In [2]:
!pip install allennlp==1.0.0 allennlp-models==1.0.0

     |████████████████████████████████| 481kB 6.1MB/s 
     |████████████████████████████████| 286kB 35.8MB/s 
     |████████████████████████████████| 675kB 32.1MB/s 
     |████████████████████████████████| 133kB 42.2MB/s 
     |████████████████████████████████| 753.2MB 23kB/s 
     |████████████████████████████████| 266kB 38.7MB/s 
     |████████████████████████████████| 317kB 42.5MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 1.1MB 47.1MB/s 
     |████████████████████████████████| 890kB 48.3MB/s 
     |████████████████████████████████| 3.8MB 31.8MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 7.0MB 38.3MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.17.0-cp36-cp36m-linux_x86_64.whl size=3387963 sha256=0f62ec13915b86de971c1753a13a9d4d2fca22d987b4d9f71ac75fb2b36c0b3f
  Stored in directory: /root/.cache/pip/wheels/26/7a/37/7dbcc30a6b4efd17b91ad1f0128b7bbf84813bd4e1cfb8c1e3
  C

In [3]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.coref
import sys
import requests
import pandas as pd

In [4]:
def download(url, filename):
	req = requests.get(url)
	url_content = req.content
	csv_file = open(filename, 'wb')
	csv_file.write(url_content)
	csv_file.close()

In [12]:
def filter(input):
    cache = []
    df = pd.read_csv(input, sep = ",")
    sentences = df["sent1"]
    #infile = open(input, 'r') 
    #lines = infile.readlines() 
    predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")
    with open(input[0:-4]+"_filtered.txt", 'w') as f:
        for _k, sent in sentences.iteritems():
        #for sent in lines:
            #if sent[-1] == "\n":
            #    sent = sent[0:-1]
            #print(sent)
            if sent not in cache:
              if len(cache) == 15:
                cache.pop(0)
              cache.append(sent)
            else: continue

            p = predictor.predict(document=sent)
            tokens = p["document"]
            clusters = p["clusters"]

            gender_indicator = 0
            for token in tokens:
              if token.lower() in male_pronoun + female_pronoun + male_signals + female_signals:
                gender_indicator += 1
            if gender_indicator != 1:
              continue

            found_pronoun = False
            select = False
            for group in clusters:
              if len(group) == 2:
                for i in range(len(group)):
                  start = group[i][0]
                  end = group[i][1]
                  if start == end:
                    word = tokens[start].lower()
                    if word in pronouns[0]+pronouns[1]:
                      found_pronoun = True
                    if found_pronoun:
                      j = (i+1) % 2
                      start2 = group[j][0]
                      end2 = group[j][1]
                      phrase = tokens[start2:end2+1]
                      if all([word.lower() != "someone" for word in phrase]):
                        select = True
                        print("masked: ", tokens[start])
                        tokens[start] = "[MASK]"

                    found_pronoun = False

                  if select: break
                if select: break
            if select:
                select = False
                sent = ' '.join(token for token in tokens) + "\n"
                f.write(sent)
                print(sent)


In [6]:
male_pronoun = ["he", "his", "him", "himself"]

female_pronoun = ["she", "her", "hers", "herself"]

male_signals = ["cowboy", "cowboys", "male", "males", "father", "grandfather", "dad", "daddy" ,"son", "grandson", "man", "men", "waiter", "waiters", "gentleman", "gentlemen", "boy", "boys", "mr", "mr.", "actor", "actors", "husband", "stepfather", "boyfriend", "brother", "brothers", "uncle", "uncles", "guy", "guys"]

female_signals = ["witch", "witches", "female", "females", "mother", "grandmother", "mom", "mum", "mommy", "daughter", "granddaughter", "woman", "women", "waitress", "waitresses", "lady", "ladies", "girl", "girls", "crone", "nun", "miss", "diva", "stepmother", "mrs", "miss.", "mrs.", "actress", "actresses", "wife", "girlfriend", "sister", "sisters", "aunt", "aunts", "beauty"]

pronouns = [male_pronoun, female_pronoun]
signals = [male_signals, female_signals]

In [7]:
download("https://raw.githubusercontent.com/rowanz/swagaf/master/data/train_full.csv", "train_full.csv")

In [ ]:
filter("train_full.csv")

masked:  his
The person holds up the violin to [MASK] chin and gets ready .

masked:  him
Someone drags the villain through a trellis and smashes [MASK] against another wall .

masked:  his
As the retired firefighter catches [MASK] breath lying face down , someone remains standing .

masked:  he
One person dives straight into water as [MASK] comes rollerblading at high speed .

masked:  his
The well - built neighbor runs out , clutching a towel around [MASK] waist .

masked:  his
The barman rolls [MASK] eyes .

masked:  his
The tall , blonde security guard checks [MASK] watch .

masked:  his
The tall , blonde security guard checks [MASK] watch .

masked:  his
A anchorman in a suit sits with a laptop on [MASK] clear desk .

masked:  his
The doctor must have been in another building , because over [MASK] shoulders is flung a dark blue overcoat .

masked:  his
The satyr meets [MASK] friends gaze and nods .

masked:  him
One of the body builders lifts the weight bar several times and then 